In [1]:
import pandas as pd
import numpy as np
from pandas import DataFrame, Series

PERIOD = 365 #used to NOT count first sales during this period -- in DAYS

In [72]:
#Load files
SALES_ORDERS = pd.read_pickle('data/temp01/SALES_ORDERS.pkl')
PARTS_MASTER = pd.read_pickle('data/temp01/PARTS_MASTER.pkl')
DISTRIB_PARTS = pd.read_pickle('data/temp01/DISTRIB_PARTS.pkl')
EXCLUD_PARTS_SO = pd.read_pickle('data/temp01/EXCLUD_PARTS_SO.pkl')
EXCLUD_CLIENT_SO = pd.read_pickle('data/temp01/EXCLUD_CLIENT_SO.pkl')
EXCLUD_SO = pd.read_pickle('data/temp01/EXCLUD_SO.pkl')

SALES_ORDERS['COMMENT'] = np.nan

#check against parts master
PARTS_MASTER.drop(columns = ['DESCRIPTION','PNM','CLASS','LIST_PRICE','PRICEDATE','CODE','MFG'],inplace = True)
PARTS_MASTER.drop_duplicates(subset = 'PN', inplace = True, keep = 'last')
SALES_ORDERS = SALES_ORDERS.merge(PARTS_MASTER, left_on = 'PN',right_on = 'PN', how = 'left')
SALES_ORDERS['COMMENT'] = np.where(SALES_ORDERS['STC'].isnull(), 'TO WARN: PN NOT IN MASTER', SALES_ORDERS['COMMENT'])

#check against excluded SO list
EXCLUD_SO.drop(columns = ['DateAdded'],inplace = True)
EXCLUD_SO.drop_duplicates(subset='SO',inplace= True)
EXCLUD_SO.rename(columns={'Comment':'COMMENT_EXC_SO','SO':'SONO'},inplace = True)
SALES_ORDERS = SALES_ORDERS.merge(EXCLUD_SO, left_on = 'SONO',right_on = 'SONO', how = 'left')

#check against excluded SO PN list 
EXCLUD_PARTS_SO.drop(columns = ['DateAdded','Description'],inplace = True)
EXCLUD_PARTS_SO.drop_duplicates(subset='PN',inplace= True)
EXCLUD_PARTS_SO.rename(columns={'Comment':'COMMENT_EXC_PN_SO'},inplace = True)
SALES_ORDERS = SALES_ORDERS.merge(EXCLUD_PARTS_SO, left_on = 'PN',right_on = 'PN', how = 'left')


#check against excluded SO Customer list 
EXCLUD_CLIENT_SO.drop(columns = ['DateAdded'],inplace = True)
EXCLUD_CLIENT_SO.drop_duplicates(subset='ClientId',inplace= True)
EXCLUD_CLIENT_SO.rename(columns={'Comment':'COMMENT_EXC_CUST_SO','ClientId':'CUSTNO'},inplace = True)
SALES_ORDERS = SALES_ORDERS.merge(EXCLUD_CLIENT_SO, left_on = 'CUSTNO',right_on = 'CUSTNO', how = 'left')


#check against Distribution list
DISTRIB_PARTS.drop(columns=['PNM','DESCRIPTION','OEM','CAGE_CODE'], inplace= True)
DISTRIB_PARTS.drop_duplicates(subset='PN',inplace = True)
DISTRIB_PARTS['IS_DISTRIB_PART'] = 1
SALES_ORDERS = SALES_ORDERS.merge(DISTRIB_PARTS, left_on = 'PN',right_on = 'PN', how = 'left')
SALES_ORDERS['IS_DISTRIB_PART'] = SALES_ORDERS['IS_DISTRIB_PART'].fillna(0)

#remove ZNEG sales
SALES_ORDERS['COMMENT'] = np.where(SALES_ORDERS['ZNEG'] == 'ZNEG', 'TO REMOVE: ZNEG', SALES_ORDERS['COMMENT'])
SALES_ORDERS['COMMENT'] = np.where(SALES_ORDERS['ZNEG'] == 'ZNEG', 'TO REMOVE: ZNEG', SALES_ORDERS['COMMENT'])

#remove first time sale within the PERIOD that are not in the distrib list
SALES_ORDERS.sort_values(by = ['ENTRY_DATE','SONO','PN'], ascending = True, inplace = True)
FIRST_TIME_SALES = SALES_ORDERS[SALES_ORDERS['IS_DISTRIB_PART'] != 1].copy()
FIRST_TIME_SALES.drop_duplicates(subset=['PN'], inplace=True,keep='first')
FIRST_TIME_SALES['COMMENT_FIRST_TIME_SALE'] = 'To remove: first time sale and not in distrib list'
FIRST_TIME_SALES.drop(columns = ['SALES_PERSON',
 'SONO',
 'SO_TYPE',
 'CUSTNO',
 'PN',
 'PNM',
 'QTYORDERED',
 'QTY_RESERVED',
 'QTYPENDING',
 'COST',
 'GEO_CODE',
 'ENTRY_DATE',
 'SHIP_DATE',
 'LEAD_TIME',
 'ZNEG',
 'DATA_SOURCE',
 'COMMENT',
 'STC',
 'COMMENT_EXC_SO',
 'COMMENT_EXC_PN_SO',
 'COMMENT_EXC_CUST_SO',
 'IS_DISTRIB_PART'],inplace = True)

SALES_ORDERS = SALES_ORDERS.merge(FIRST_TIME_SALES,left_index = True, right_index=True,how='left')

#TODO : pull out all excluded rows on dashboad

SALES_ORDERS['EXCLUDED'] = np.where((SALES_ORDERS['COMMENT'] != 'nan') |
                                    (~SALES_ORDERS['COMMENT_EXC_SO'].isnull()) |
                                    (~SALES_ORDERS['COMMENT_EXC_PN_SO'].isnull()) |
                                    (~SALES_ORDERS['COMMENT_EXC_CUST_SO'].isnull()) |
                                    (~SALES_ORDERS['COMMENT_FIRST_TIME_SALE'].isnull()), True, False)


SALES_ORDERS.to_pickle('data/temp02/SALES_ORDERS.pkl')